In [1]:
import os
import multiprocessing
import numpy as np
import pandas as pd
from sklearn import linear_model, metrics, model_selection
from sys import argv

In [3]:
dat_in_pth = "../data/features.feather"
dat_df = pd.read_feather(dat_in_pth).set_index(keys="index")

In [4]:
dat_df.head()

,M0004_1.02_FWD_1,M0004_1.02_FWD_2,M0004_1.02_FWD_3,M0004_1.02_FWD_4,M0004_1.02_FWD_5,M0004_1.02_FWD_6,M0004_1.02_FWD_7,M0005_1.02_FWD_1,M0005_1.02_FWD_2,M0005_1.02_FWD_3,...,GA_REV_2,GA_REV_3,GA_REV_4,GA_REV_5,GA_REV_6,GA_REV_7,GCcontent,CAcontent,GAcontent,class
index,,,,,,,,,,,,,,,,,,,,,
Chr4_+_7750398_AT4G13310_root_0,1.440441,0.000000,0.000000,4.508889,9.438149,9.714371,17.533581,5.501110,0.0,0.0,...,11.805773,9.450124,12.967102,8.974781,14.424215,13.597577,0.300,0.590,0.580,1
Chr4_+_11302680_AT4G21200_leaf_0,11.111250,11.111250,0.000000,0.000000,12.846526,16.472290,5.290287,2.406347,0.0,0.0,...,0.824729,9.927819,16.496307,6.568488,4.865940,4.865940,0.340,0.540,0.430,1
Chr4_-_10918262_AT4G20210_root_0,0.293377,4.510425,4.217049,4.559656,5.156968,0.597312,0.000000,0.000000,0.0,0.0,...,2.222053,11.242960,9.343733,6.638287,25.419486,18.781199,0.410,0.545,0.475,1
Chr5_-_26757236_AT5G67030_tair_0,0.555420,2.275065,2.275065,0.000000,0.000000,0.189317,22.804014,0.000000,0.0,0.0,...,9.204328,3.095468,0.000000,11.282284,13.337318,11.061596,0.425,0.495,0.440,1
Chr1_-_25046860_AT1G67080_tair_0,0.000000,1.398959,1.398959,1.377300,2.470973,1.093673,0.000000,0.000000,0.0,0.0,...,11.599996,10.826384,11.891774,11.529227,11.948718,8.528950,0.305,0.545,0.550,1


In [ ]:
results_dir = "results"
seed = 8675309
np.random.seed(seed)

seed_out_pth = results_dir + "/" + str(seed)
if not os.path.exists(seed_out_pth):
    os.makedirs(seed_out_pth)

training_set_tss_ids_pth = seed_out_pth + "/training_set_tss_ids.txt"
heldout_test_set_tss_ids_pth = seed_out_pth + "/heldout_test_set_tss_ids.txt"
crossval_results_pth = seed_out_pth + "/crossval_results.csv"
heldout_test_performance_pth = seed_out_pth + "/heldout_test_performance.csv"
factors_pth = seed_out_pth + "/factors.csv"

###########################################################################
# split data into 80/20 train/test
train, heldout_test = model_selection.train_test_split(
    dat_df, test_size=0.2, stratify=dat_df["class"], random_state=seed
)

np.savetxt(training_set_tss_ids_pth, train.index.values, fmt="%s")
np.savetxt(heldout_test_set_tss_ids_pth, heldout_test.index.values, fmt="%s")

###########################################################################
# define log reg cv object
log_cv = linear_model.LogisticRegressionCV(
    solver="liblinear",
    Cs=np.logspace(-4, 4, 100),
    cv=5,
    penalty="l1",
    n_jobs=int(multiprocessing.cpu_count() / 2),
    refit=True,
    scoring="average_precision",
    random_state=seed,
)

# train model
log_cv.fit(train.iloc[:, :-1], y=train["class"])

###########################################################################
# test final model on heldout test set
y_true, y_prediction_probabilities = (
    heldout_test["class"],
    log_cv.predict_proba(heldout_test.iloc[:, :-1]),
)

###########################################################################
# get various metrics and save to disk
roc_curve_df = (
    pd.DataFrame(metrics.roc_curve(y_true, y_prediction_probabilities[:, 1]))
    .T.rename(columns={0: "FPR", 1: "TPR", 2: "threshold"})
    .drop(index=0)
    .reset_index(drop=True)
)
roc_curve_df["Y"] = roc_curve_df["TPR"] - roc_curve_df["FPR"]
roc_curve_df = roc_curve_df.round(4)
youden_T = roc_curve_df.iloc[roc_curve_df["Y"].idxmax()]["threshold"]

y_pred_youden = np.where(y_prediction_probabilities[:, 1] >= youden_T, 1, 0)

heldout_perf_roc = metrics.roc_auc_score(y_true, y_prediction_probabilities[:, 1])
heldout_perf_prc = metrics.average_precision_score(
    y_true, y_prediction_probabilities[:, 1]
)

confusion_mat = metrics.confusion_matrix(y_true, y_pred_youden)
TN = confusion_mat[0][0]
FP = confusion_mat[0][1]
FN = confusion_mat[1][0]
TP = confusion_mat[1][1]

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP / (TP + FN)
# Specificity or true negative rate
TNR = TN / (TN + FP)
# Precision or positive predictive value
PPV = TP / (TP + FP)

heldout_perf_f1 = 2 * ((PPV * TPR) / (PPV + TPR))

metrics_dict = {
    "C": log_cv.C_[0],
    "auROC": heldout_perf_roc,
    "auPRC": heldout_perf_prc,
    "youden_T": youden_T,
    "sensitivity": TPR,
    "specificity": TNR,
    "precision": PPV,
    "f1": heldout_perf_f1,
    "seed": seed,
    "ratio": neg_to_pos_ratio,
}

# save metrics as csv
metrics_df = pd.DataFrame(
    metrics_dict.values(), index=metrics_dict.keys(), dtype=float
).T
metrics_df["seed"] = metrics_df["seed"].astype(int)
metrics_df.to_csv(heldout_test_performance_pth, header=True)

# save coefficient values
factors_df = pd.DataFrame(log_cv.coef_, columns=dat_df.columns[:-1])
factors_df["seed"] = seed
factors_df["ratio"] = neg_to_pos_ratio
factors_df.to_csv(factors_pth, header=True)
